# Régression pour retrouver le temps de livraison

Ruben SAILLY, Frédéric EGENSCHEVILLER

Chapitrage :
- [Installation des packages nécessaires](#-installation-des-packages-nécessaires)
- [1. Import des données et visualisation](#-1.-import-des-données-et-visualisation)
- [2. Préprocessing avec pipeline scikit-learn](#-2.-préprocessing-avec-pipeline-scikit-learn)

## Installation des packages nécessaires

In [1]:
from sklearn.linear_model import LinearRegression
!pip install pandas numpy scikit-learn matplotlib seaborn kagglehub


[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


## 1. Import des données et visualisation

On va télécharger le dataset via kagglehub, puis on va visualiser les données, afin de voir ce que l'on va devoir faire pour les nettoyer.
Le nettoyage se fera dans un second temps avec une pipeline scikit-learn dédiée.

In [2]:
import kagglehub
import pandas as pd
path = kagglehub.dataset_download("gauravmalik26/food-delivery-dataset")
print("Path to dataset files:", path)

df_test = pd.read_csv(path + "/test.csv")
df_train = pd.read_csv(path + "/train.csv")

/Users/rubensailly/Documents/IMT/FIL_A3/ML/final_project/.venv/lib/python3.14/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


100%|██████████| 1.95M/1.95M [00:00<00:00, 4.26MB/s]

Extracting files...
Path to dataset files: /Users/rubensailly/.cache/kagglehub/datasets/gauravmalik26/food-delivery-dataset/versions/1


Les fichiers téléchargés sont les suivants :
- ```test.csv```: le dataset de test, avec les features mais sans la target (temps de livraison)
- ```train.csv```: le dataset d'entrainement, avec les features et la target (temps de livraison)
- ```sample_submission.csv```: un exemple de fichier de soumission pour kaggle, avec ids de ```test.csv``` et des valeurs quelconques pour la target.

On va donc utiliser ```train.csv```, étant donné que c'est le seul qui contient la target, et que ```test.csv``` sert à évaluer notre modèle sur kaggle.

Regardons les données d'entrainement.

In [3]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45593 entries, 0 to 45592
Data columns (total 20 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   ID                           45593 non-null  object 
 1   Delivery_person_ID           45593 non-null  object 
 2   Delivery_person_Age          45593 non-null  object 
 3   Delivery_person_Ratings      45593 non-null  object 
 4   Restaurant_latitude          45593 non-null  float64
 5   Restaurant_longitude         45593 non-null  float64
 6   Delivery_location_latitude   45593 non-null  float64
 7   Delivery_location_longitude  45593 non-null  float64
 8   Order_Date                   45593 non-null  object 
 9   Time_Orderd                  45593 non-null  object 
 10  Time_Order_picked            45593 non-null  object 
 11  Weatherconditions            45593 non-null  object 
 12  Road_traffic_density         45593 non-null  object 
 13  Vehicle_conditio

On voit que les données sont majoritairement des chaînes de caractères (object), même la target. Regardons les premières lignes du dataset pour toutes les colonnes, afin de distinguer :
- ce qui est numérique
- ce qui est catégoriel
- ce qui est textuel

Comme on a presque que des colonnes ```(object)```, si un NaN est présent dans la colonne, il n'est pas traité comme tel, mais comme une chaîne de caractères ```'NaN'```. Il faudra faire attention à ça lors du nettoyage. Essayons de compter le nombre de chaines de caractères "NaN" dans chaque colonne :

In [4]:
for col in df_train.columns:
    print(f"Colonne {col} : {df_train[col].value_counts().get('NaN ', 0)} valeurs 'NaN'")

Colonne ID : 0 valeurs 'NaN'
Colonne Delivery_person_ID : 0 valeurs 'NaN'
Colonne Delivery_person_Age : 1854 valeurs 'NaN'
Colonne Delivery_person_Ratings : 1908 valeurs 'NaN'
Colonne Restaurant_latitude : 0 valeurs 'NaN'
Colonne Restaurant_longitude : 0 valeurs 'NaN'
Colonne Delivery_location_latitude : 0 valeurs 'NaN'
Colonne Delivery_location_longitude : 0 valeurs 'NaN'
Colonne Order_Date : 0 valeurs 'NaN'
Colonne Time_Orderd : 1731 valeurs 'NaN'
Colonne Time_Order_picked : 0 valeurs 'NaN'
Colonne Weatherconditions : 0 valeurs 'NaN'
Colonne Road_traffic_density : 601 valeurs 'NaN'
Colonne Vehicle_condition : 0 valeurs 'NaN'
Colonne Type_of_order : 0 valeurs 'NaN'
Colonne Type_of_vehicle : 0 valeurs 'NaN'
Colonne multiple_deliveries : 993 valeurs 'NaN'
Colonne Festival : 228 valeurs 'NaN'
Colonne City : 1200 valeurs 'NaN'
Colonne Time_taken(min) : 0 valeurs 'NaN'


On va donc devoir traiter les ```'NaN '``` comme des valeurs manquantes dans notre fonction de nettoyage. Cela veut dire que l'on va devoir utiliser des imputeurs dans notre pipeline scikit-learn.

In [5]:
df_train.head()

,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Order_Date,Time_Orderd,Time_Order_picked,Weatherconditions,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City,Time_taken(min)
0,0x4607,INDORES13DEL02,37,4.9,22.745049,75.892471,22.765049,75.912471,19-03-2022,11:30:00,11:45:00,conditions Sunny,High,2,Snack,motorcycle,0,No,Urban,(min) 24
1,0xb379,BANGRES18DEL02,34,4.5,12.913041,77.683237,13.043041,77.813237,25-03-2022,19:45:00,19:50:00,conditions Stormy,Jam,2,Snack,scooter,1,No,Metropolitian,(min) 33
2,0x5d6d,BANGRES19DEL01,23,4.4,12.914264,77.678400,12.924264,77.688400,19-03-2022,08:30:00,08:45:00,conditions Sandstorms,Low,0,Drinks,motorcycle,1,No,Urban,(min) 26
3,0x7a6a,COIMBRES13DEL02,38,4.7,11.003669,76.976494,11.053669,77.026494,05-04-2022,18:00:00,18:10:00,conditions Sunny,Medium,0,Buffet,motorcycle,1,No,Metropolitian,(min) 21
4,0x70a2,CHENRES12DEL01,32,4.6,12.972793,80.249982,13.012793,80.289982,26-03-2022,13:30:00,13:45:00,conditions Cloudy,High,1,Snack,scooter,1,No,Metropolitian,(min) 30


In [6]:
df_train.nunique()

ID                             45593
Delivery_person_ID              1320
Delivery_person_Age               23
Delivery_person_Ratings           29
Restaurant_latitude              657
Restaurant_longitude             518
Delivery_location_latitude      4373
Delivery_location_longitude     4373
Order_Date                        44
Time_Orderd                      177
Time_Order_picked                193
Weatherconditions                  7
Road_traffic_density               5
Vehicle_condition                  4
Type_of_order                      4
Type_of_vehicle                    4
multiple_deliveries                5
Festival                           3
City                               4
Time_taken(min)                   45
dtype: int64

On peut déjà affirmer les choses suivantes :
- ```ID``` est une chaine de caractères, et est un nombre hexadécimal. Il est unique pour chaque ligne, donc il ne servira pas pour la régression, on le supprimera.
- ```Delivery_person_ID``` est aussi une chaine de caractères, mais elle n'est pas unique, donc elle peut servir pour la régression. C'est une variable catégorielle.
- ```Delivery_person_Age``` est numérique, mais est stockée en tant que chaîne de caractères. Il faudra la convertir en numérique.
- ```Delivery_person_Ratings``` est numérique, mais est stockée en tant que chaîne de caractères. Il faudra la convertir en numérique.
- ```Restaurant_latitude```, ```Restaurant_longitude```, ```Delivery_location_latitude```, ```Delivery_location_longitude``` sont des coordonnées GPS, stockées en numérique. On pourra en déduire la distance entre le restaurant et le lieu de livraison, qui sera une feature intéressante.
- ```Order_Date``` est une date, stockée en tant que chaîne de caractères. On pourra en extraire des informations intéressantes, comme le jour de la semaine, le mois, etc.
- ```Time_Orderd``` et ```Time_Order_picked``` sont des heures, stockées en tant que chaînes de caractères. On pourra en extraire des informations intéressantes, comme l'heure de la journée. Étant donné que l'on a pas de valeurs manquantes dans ces colonnes, et que il y a dans chaque colonne 200 valeurs distinctes, on peut les traiter comme des variables catégorielles.
- ```Weatherconditions```, ```Road_traffic_density```, ```Type_of_order```, ```Type_of_vehicle```, ```Festival```, ```City``` sont des variables catégorielles, stockées en tant que chaînes de caractères.
- ```multiple_deliveries``` est numérique, mais est stockée en tant que chaîne de caractères. Il faudra la convertir en numérique.
- ```Time_taken(min)``` est la target, elle est numérique mais stockée en tant que chaîne de caractères. Il faudra la convertir en numérique. On va devoir faire une fonction de nettoyage pour nettoyer les ```(min)```présents dans les valeurs.

Essayons de voir si ```(min)``` est présent dans toutes les lignes de la target, et est ce que l'on retrouve d'autres chaînes de caractères dans cette colonne.

In [7]:
df_train["Time_taken(min)"].unique()

array(['(min) 24', '(min) 33', '(min) 26', '(min) 21', '(min) 30',
       '(min) 40', '(min) 32', '(min) 34', '(min) 46', '(min) 23',
       '(min) 20', '(min) 41', '(min) 15', '(min) 36', '(min) 39',
       '(min) 18', '(min) 38', '(min) 47', '(min) 12', '(min) 22',
       '(min) 25', '(min) 35', '(min) 10', '(min) 19', '(min) 11',
       '(min) 28', '(min) 52', '(min) 16', '(min) 27', '(min) 49',
       '(min) 17', '(min) 14', '(min) 37', '(min) 44', '(min) 42',
       '(min) 31', '(min) 13', '(min) 29', '(min) 50', '(min) 43',
       '(min) 48', '(min) 54', '(min) 53', '(min) 45', '(min) 51'],
      dtype=object)

On peut voir que toutes les valeurs contiennent ```(min)```, le nettoyage est donc simple, on va juste devoir supprimer cette partie de la chaîne de caractères, puis convertir en numérique.

## 2. Préprocessing avec pipeline scikit-learn

### 2.1 Fonction de nettoyage

On va commencer par faire notre fonction de nettoyage avant les encodeurs et les scalers. Il faut faire attention à traiter les ```NaN ``` comme des veleurs manquantes.
De plus, on va ajouter une colonne ```Computed_distance```, un float qui contiendra la distance entre le restaurant et le lieu de livraison, calculée à partir des coordonnées GPS. Ce n'est pas la distance exacte (distance à vol d'oiseau), mais ça donnera une idée de la distance, et ça pourra aider le modèle. On pourrait à terme utiliser une API de calcul d'itinéraire, mais pour l'instant on va se contenter de ça.

Nous allons modifier les colonnes de date et heure pour ne pas les avoir en catégorielles, mais en numérique. On va avoir à partir des clones ```Order_Date```, ```Time_Orderd``` et ```Time_Order_picked``` les nouvelles colonnes suivantes :
- ```Order_Week_Day```: le jour de la semaine (0 = lundi, 6 = dimanche)
- ```Order_Time_Min```: le nombre de minutes depuis minuit pour l'heure de la commande
- ```Order_Picked_Time_Min```: le nombre de minutes depuis minuit pour l'heure de la prise en charge de la commande
- ```Order_DaysSinceFirstOrder```: le nombre de jours depuis la première commande dans le dataset (pour avoir une notion de temporalité)

In [95]:
import numpy as np
import pandas as pd
from math import radians, sin, cos, sqrt, atan2
from sklearn.base import BaseEstimator, TransformerMixin
from datetime import datetime

pd.set_option('future.no_silent_downcasting', True)

class PreCleaner(BaseEstimator, TransformerMixin):

    def __init__(self, df: pd.DataFrame = None, target_col: str = "Time_taken(min)", numeric_cols=None,
                 boolean_cols=None, categorical_cols=None):
        if numeric_cols is None:
            numeric_cols = ["Delivery_person_Age", "Delivery_person_Ratings", "multiple_deliveries", "Order_Week_Day", "Order_Time_Min", "Order_Picked_Time_Min", "Order_DaysSinceFirstOrder", "Computed_distance"]
        if boolean_cols is None:
            boolean_cols = ["Festival"]
        if categorical_cols is None:
            categorical_cols = ["Delivery_person_ID", "Weatherconditions", "Road_traffic_density","Type_of_order", "Type_of_vehicle", "City"]
        self.numeric_cols = numeric_cols
        self.boolean_cols = boolean_cols
        self.categorical_cols = categorical_cols
        self.df = df
        self.target_col = target_col


    def cleanTarget(self, df: pd.DataFrame, target_col: str = "Time_taken(min)") -> pd.DataFrame:
        df[target_col] = df[target_col].str.replace("(min) ", "").infer_objects(copy=False) .astype(np.int32)
        return df

    def cleanNumericCols(self, df: pd.DataFrame, numeric_cols: list) -> pd.DataFrame:
        for col in numeric_cols:
            df[col] = df[col].replace({'NaN ': np.nan}).infer_objects(copy=False)
            df[col] = pd.to_numeric(df[col], errors='coerce')
        return df

    def cleanBooleanCols(self, df: pd.DataFrame, boolean_cols=None) -> pd.DataFrame:
        if boolean_cols is None:
            boolean_cols = ["Festival"]
        for col in boolean_cols:
            df[col] = df[col].replace({'Yes ': 1, 'No ': 0, 'NaN ': np.nan}).infer_objects(copy=False)
        return df

    def cleanCategoricalCols(self, df: pd.DataFrame, categorical_cols=None) -> pd.DataFrame:
        if categorical_cols is None:
            categorical_cols = ["ID", "Delivery_person_ID", "Weatherconditions", "Road_traffic_density","Type_of_order", "Type_of_vehicle", "City", "Time_Orderd", "Time_Order_picked", "Order_Date", "Vehicle_condition"]
        for col in categorical_cols:
            df[col] = df[col].replace({'NaN ': np.nan}).infer_objects(copy=False)
        return df

    def date_parser(self,
                    df: pd.DataFrame,
                    time_orderd: str = "Time_Orderd",
                    time_orderd_picked: str = "Time_Order_picked",
                    date_col: str = "Order_Date") -> pd.DataFrame:
        # Le but est de convertir les colonnes Time_Orderd et Time_Orderd_picked en datetime pour avoir des données exhaustives:
        # - jour de la semaine
        # - nb de minutes depuis minuit (en int pour les algos de régression que l'on va utiliser)
        # - nb de jours depuis le plus ancien date dans le dataset (en int pour les algos de régression que l'on va utiliser)
        Order_Week_Day = []
        Order_Time_Min = []
        Order_Picked_Time_Min = []
        Order_DaysSinceFirstOrder = []
        for col in [time_orderd, time_orderd_picked, date_col]:
            df[col] = df[col].replace({'NaN ': np.nan}).infer_objects(copy=False)
        minimal_date = pd.to_datetime(df[date_col], format="%d-%m-%Y").min()
        for _, row in df.iterrows():
            Order_Week_Day.append(pd.to_datetime(row[date_col], format="%d-%m-%Y").weekday())
            order_timestamp = pd.to_datetime(row[time_orderd], format="%H:%M:%S")
            Order_Time_Min.append(order_timestamp.hour * 60 + order_timestamp.minute)
            order_picked_timestamp = pd.to_datetime(row[time_orderd_picked], format="%H:%M:%S")
            Order_Picked_Time_Min.append(order_picked_timestamp.hour * 60 + order_picked_timestamp.minute)
            days_since_first_order = (pd.to_datetime(row[date_col], format="%d-%m-%Y") - minimal_date).days
            Order_DaysSinceFirstOrder.append(days_since_first_order)
        df["Order_Week_Day"] = Order_Week_Day
        df["Order_Time_Min"] = Order_Time_Min
        df["Order_Picked_Time_Min"] = Order_Picked_Time_Min
        df["Order_DaysSinceFirstOrder"] = Order_DaysSinceFirstOrder
        return df

    def computeDistance(self, df: pd.DataFrame,
                        lat1_col: str = "Restaurant_latitude",
                        lon1_col: str = "Restaurant_longitude",
                        lat2_col: str = "Delivery_location_latitude",
                        lon2_col: str = "Delivery_location_longitude") -> pd.DataFrame:
        def haversine(lat1, lon1, lat2, lon2):
            R = 6371.0  # Rayon de la Terre en kilomètres
            dlat = radians(lat2 - lat1)
            dlon = radians(lon2 - lon1)
            a = sin(dlat / 2)**2 + cos(radians(lat1)) * cos(radians(lat2)) * sin(dlon / 2)**2
            c = 2 * atan2(sqrt(a), sqrt(1 - a))
            distance = R * c
            return distance
        distances = []
        for _, row in df.iterrows():
            distance = haversine(row[lat1_col], row[lon1_col], row[lat2_col], row[lon2_col])
            distances.append(distance)
        df["Computed_distance"] = distances
        return df

    def dropColons(self, df: pd.DataFrame, droped_colons=None) -> pd.DataFrame:
        if droped_colons is None:
            droped_colons = ["ID", "Restaurant_latitude", "Restaurant_longitude",
                             "Delivery_location_latitude", "Delivery_location_longitude",
                             "Order_Date", "Time_Orderd", "Time_Order_picked"]
        df = df.drop(columns=droped_colons)
        return df

    def fit(self, X, y=None):
        return self

    def transform(self, X: pd.DataFrame = None) -> pd.DataFrame:
        if X is not None:
            self.df = X
        df_cp = X.copy()
        df_cp = self.date_parser(df_cp)
        df_cp = self.computeDistance(df_cp)
        # df_cp = self.cleanTarget(df_cp, self.target_col)
        df_cp = self.cleanNumericCols(df_cp, self.numeric_cols)
        df_cp = self.cleanBooleanCols(df_cp, self.boolean_cols)
        df_cp = self.cleanCategoricalCols(df_cp, self.categorical_cols)
        df_cp = self.dropColons(df_cp)
        return df_cp

    def get_target_col(self):
        return self.target_col

    def get_numeric_cols(self):
        return self.numeric_cols + self.boolean_cols

    def get_categorical_cols(self):
        return self.categorical_cols

    def get_dataframe(self):
        return self.df

### 2.2 Pipeline scikit-learn

On vient de faire une classe de pré-nettoyage, on va maintenant intégrer cette classe dans une pipeline scikit-learn, afin de pouvoir l'utiliser facilement avec des encodeurs et des scalers.

De plus, on peut remarquer plus haut que dans les colonnes catégoriques, certaines colonnes ont beaucoup de modalités, d'autres en ont peu.

On va donc utiliser un ```OneHotEncoder``` pour les colonnes avec peu de modalités, et un ```TargetEncoder``` pour les colonnes avec beaucoup de modalités. Cela permettra de réduire la dimensionnalité du dataset après encodage, et d'éviter le problème de la malédiction de la dimensionnalité.

Pour les colonnes numériques, on va utiliser un ```StandardScaler``` pour les mettre à l'échelle. La colonne booléenne n'a pas besoin d'être mise à l'échelle, car elle est déjà binaire, et sera laissée telle quelle.

Pour les imputeurs, on va utiliser un ```SimpleImputer``` avec la stratégie de la moyenne pour les colonnes numériques, et la stratégie de la modalité la plus fréquente pour les colonnes catégorielles. On changera peut être cela plus tard, en fonction des résultats obtenus.

In [96]:
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, TargetEncoder

cleaner = PreCleaner(df_train, target_col="Time_taken(min)")

#pre_cleaner = FunctionTransformer(func= lambda df, y=None: cleaner.transform(df), validate=False)

"""
numerical_cols = ["Delivery_person_Age", "Delivery_person_Ratings", "multiple_deliveries", "Computed_distance", "Festival", "Time_taken(min)"]
categorical_cols = ["Delivery_person_ID", "Weatherconditions", "Road_traffic_density",
                    "Type_of_order", "Type_of_vehicle", "City", "Time_Orderd", "Time_Order_picked",
                    "Order_Date", "Vehicle_condition"]
target = "Time_taken(min)"
"""

numerical_cols = cleaner.get_numeric_cols()
categorical_cols = cleaner.get_categorical_cols()
target = cleaner.get_target_col()

numerical_Pipeline = Pipeline(steps=[
    ('imputer',  SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

low_modalities_cols = [col for col in categorical_cols if df_train[col].nunique() < 20]
high_modalities_cols = [col for col in categorical_cols if df_train[col].nunique() >= 20]

categorical_low_Pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

categorical_high_Pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('target_encoder',  TargetEncoder())
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_Pipeline, numerical_cols),
        ('cat_low', categorical_low_Pipeline, low_modalities_cols),
        ('cat_high', categorical_high_Pipeline, high_modalities_cols)
    ])

full_pipeline = Pipeline(steps=[
    ('pre_cleaner', PreCleaner()),
    ('preprocessor', preprocessor)
])

Voilà notre pipeline de preprocessing complète :

In [86]:
full_pipeline

,steps,"[('pre_cleaner', ...), ('preprocessor', ...)]"
,transform_input,None
,memory,None
,verbose,False
,df,None
,target_col,'Time_taken(min)'
,numeric_cols,"['Delivery_person_Age', 'Delivery_person_Ratings', ...]"
,boolean_cols,['Festival']
,categorical_cols,"['Delivery_person_ID', 'Weatherconditions', ...]"
,transformers,"[('num', ...), ('cat_low', ...), ...]"
,remainder,'drop'


In [97]:
from sklearn.linear_model import LinearRegression
estimation_pipeline = Pipeline([
    ('preprocessing', full_pipeline),
    ('regressor', LinearRegression())
])

In [108]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Ridge, Lasso, ElasticNet, SGDRegressor, BayesianRidge
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, HistGradientBoostingRegressor

param_grid = [
    {'regressor': [LinearRegression()], 'regressor__fit_intercept': [True, False]},
    {'regressor': [Ridge()], 'regressor__alpha': [0.1, 1.0, 10.0]},
    {'regressor': [Lasso(max_iter=10000)], 'regressor__alpha': [0.001, 0.01, 0.1, 0.5], 'regressor__selection': ['cyclic', 'random'], 'regressor__tol': [1e-4, 1e-3]},
    {'regressor': [ElasticNet(max_iter=5000)], 'regressor__alpha': [0.01, 0.1, 1.0], 'regressor__l1_ratio': [0.2, 0.5, 0.8]},
    {'regressor': [SGDRegressor(max_iter=2000, tol=1e-3)], 'regressor__alpha': [1e-4, 1e-3]},
    {'regressor': [BayesianRidge()], },
    {"regressor": [RandomForestRegressor(random_state=42)], "regressor__n_estimators": [200, 500],
     "regressor__max_depth": [None, 12], "regressor__min_samples_leaf": [1, 3]},
    {"regressor": [GradientBoostingRegressor(random_state=42)], "regressor__n_estimators": [300],
     "regressor__learning_rate": [0.05, 0.1], "regressor__max_depth": [3, 5, 7]},
    {"regressor": [HistGradientBoostingRegressor(random_state=42)],
     "regressor__learning_rate": [0.05, 0.1], "regressor__max_leaf_nodes": [31, 63, 127]}
]

grid = GridSearchCV(estimation_pipeline, param_grid, cv=5, scoring='neg_mean_absolute_error', n_jobs=-1, verbose=3, error_score='raise')

In [99]:
cleaner.transform(df_train)

,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Weatherconditions,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City,Time_taken(min),Order_Week_Day,Order_Time_Min,Order_Picked_Time_Min,Order_DaysSinceFirstOrder,Computed_distance
0,INDORES13DEL02,37.0,4.9,conditions Sunny,High,2,Snack,motorcycle,0.0,0,Urban,24,5,690.0,705,36,3.025149
1,BANGRES18DEL02,34.0,4.5,conditions Stormy,Jam,2,Snack,scooter,1.0,0,Metropolitian,33,4,1185.0,1190,42,20.183530
2,BANGRES19DEL01,23.0,4.4,conditions Sandstorms,Low,0,Drinks,motorcycle,1.0,0,Urban,26,5,510.0,525,36,1.552758
3,COIMBRES13DEL02,38.0,4.7,conditions Sunny,Medium,0,Buffet,motorcycle,1.0,0,Metropolitian,21,1,1080.0,1090,53,7.790401
4,CHENRES12DEL01,32.0,4.6,conditions Cloudy,High,1,Snack,scooter,1.0,0,Metropolitian,30,5,810.0,825,43,6.210138
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45588,JAPRES04DEL01,30.0,4.8,conditions Windy,High,1,Meal,motorcycle,0.0,0,Metropolitian,32,3,695.0,705,41,1.489846
45589,AGRRES16DEL01,21.0,4.6,conditions Windy,Jam,0,Buffet,motorcycle,1.0,0,Metropolitian,36,2,1195.0,1210,5,11.007735
45590,CHENRES08DEL03,30.0,4.9,conditions Cloudy,Low,1,Drinks,scooter,0.0,0,Metropolitian,16,4,1430.0,5,28,4.657195
45591,COIMBRES11DEL01,20.0,4.7,conditions Cloudy,High,0,Snack,motorcycle,1.0,0,Metropolitian,26,0,815.0,820,24,6.232393


In [100]:
cleaner.get_dataframe()

,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Order_Date,Time_Orderd,Time_Order_picked,Weatherconditions,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City,Time_taken(min)
0,0x4607,INDORES13DEL02,37,4.9,22.745049,75.892471,22.765049,75.912471,19-03-2022,11:30:00,11:45:00,conditions Sunny,High,2,Snack,motorcycle,0,No,Urban,24
1,0xb379,BANGRES18DEL02,34,4.5,12.913041,77.683237,13.043041,77.813237,25-03-2022,19:45:00,19:50:00,conditions Stormy,Jam,2,Snack,scooter,1,No,Metropolitian,33
2,0x5d6d,BANGRES19DEL01,23,4.4,12.914264,77.678400,12.924264,77.688400,19-03-2022,08:30:00,08:45:00,conditions Sandstorms,Low,0,Drinks,motorcycle,1,No,Urban,26
3,0x7a6a,COIMBRES13DEL02,38,4.7,11.003669,76.976494,11.053669,77.026494,05-04-2022,18:00:00,18:10:00,conditions Sunny,Medium,0,Buffet,motorcycle,1,No,Metropolitian,21
4,0x70a2,CHENRES12DEL01,32,4.6,12.972793,80.249982,13.012793,80.289982,26-03-2022,13:30:00,13:45:00,conditions Cloudy,High,1,Snack,scooter,1,No,Metropolitian,30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45588,0x7c09,JAPRES04DEL01,30,4.8,26.902328,75.794257,26.912328,75.804257,24-03-2022,11:35:00,11:45:00,conditions Windy,High,1,Meal,motorcycle,0,No,Metropolitian,32
45589,0xd641,AGRRES16DEL01,21,4.6,0.000000,0.000000,0.070000,0.070000,16-02-2022,19:55:00,20:10:00,conditions Windy,Jam,0,Buffet,motorcycle,1,No,Metropolitian,36
45590,0x4f8d,CHENRES08DEL03,30,4.9,13.022394,80.242439,13.052394,80.272439,11-03-2022,23:50:00,00:05:00,conditions Cloudy,Low,1,Drinks,scooter,0,No,Metropolitian,16
45591,0x5eee,COIMBRES11DEL01,20,4.7,11.001753,76.986241,11.041753,77.026241,07-03-2022,13:35:00,13:40:00,conditions Cloudy,High,0,Snack,motorcycle,1,No,Metropolitian,26


In [101]:
df_train[target]

0        24
1        33
2        26
3        21
4        30
         ..
45588    32
45589    36
45590    16
45591    26
45592    36
Name: Time_taken(min), Length: 45593, dtype: int32

In [109]:
#grid.fit(df_train.drop(columns=[target]), df_train[target])
grid.fit(df_train.drop(columns=[target]), df_train[target])

Fitting 5 folds for each of 53 candidates, totalling 265 fits


/var/folders/85/n4jxcw0929d3vrn7d5v4914r0000gn/T/ipykernel_53865/2361337285.py:40: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
/var/folders/85/n4jxcw0929d3vrn7d5v4914r0000gn/T/ipykernel_53865/2361337285.py:40: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
/var/folders/85/n4jxcw0929d3vrn7d5v4914r0000gn/T/ipykernel_53865/2361337285.py:40: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to th

[CV 1/5] END regressor=LinearRegression(), regressor__fit_intercept=False;, score=-5.039 total time=  11.1s
[CV 5/5] END regressor=LinearRegression(), regressor__fit_intercept=True;, score=-5.031 total time=  11.3s
[CV 3/5] END regressor=LinearRegression(), regressor__fit_intercept=True;, score=-5.025 total time=  11.4s
[CV 4/5] END regressor=LinearRegression(), regressor__fit_intercept=True;, score=-5.012 total time=  11.4s


/var/folders/85/n4jxcw0929d3vrn7d5v4914r0000gn/T/ipykernel_53865/2361337285.py:40: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
/var/folders/85/n4jxcw0929d3vrn7d5v4914r0000gn/T/ipykernel_53865/2361337285.py:40: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`


[CV 2/5] END regressor=LinearRegression(), regressor__fit_intercept=True;, score=-5.057 total time=  11.5s
[CV 1/5] END regressor=LinearRegression(), regressor__fit_intercept=True;, score=-5.038 total time=  11.5s


...

/var/folders/85/n4jxcw0929d3vrn7d5v4914r0000gn/T/ipykernel_53865/2361337285.py:40: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
/var/folders/85/n4jxcw0929d3vrn7d5v4914r0000gn/T/ipykernel_53865/2361337285.py:40: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`


[CV 5/5] END regressor=HistGradientBoostingRegressor(random_state=42), regressor__learning_rate=0.1, regressor__max_leaf_nodes=127;, score=-3.492 total time=  16.0s
[CV 4/5] END regressor=GradientBoostingRegressor(random_state=42), regressor__learning_rate=0.1, regressor__max_depth=7, regressor__n_estimators=300;, score=-3.535 total time= 3.3min


/var/folders/85/n4jxcw0929d3vrn7d5v4914r0000gn/T/ipykernel_53865/2361337285.py:40: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`


[CV 5/5] END regressor=GradientBoostingRegressor(random_state=42), regressor__learning_rate=0.1, regressor__max_depth=7, regressor__n_estimators=300;, score=-3.530 total time= 3.0min


,estimator,Pipeline(step...egression())])
,param_grid,"[{'regressor': [LinearRegression()], 'regressor__fit_intercept': [True, False]}, {'regressor': [Ridge()], 'regressor__alpha': [0.1, 1.0, ...]}, ...]"
,scoring,'neg_mean_absolute_error'
,n_jobs,-1
,refit,True
,cv,5
,verbose,3
,pre_dispatch,'2*n_jobs'
,error_score,'raise'
,return_train_score,False
,steps,"[('pre_cleaner', ...), ('preprocessor', ...)]"


In [110]:
grid.best_params_, grid.best_score_

({'regressor': HistGradientBoostingRegressor(random_state=42),
  'regressor__learning_rate': 0.05,
  'regressor__max_leaf_nodes': 127},
 np.float64(-3.4664816580772615))